In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# This is the second Notebook for the ecommerce Marketing Data. Please check the first notebook if the next line is confusing. 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
data=pd.read_csv('../input/marketing-data/marketing_data.csv')
data[' Income '].loc[data.loc[(data.Education=='PhD') & (data[' Income '].isna())].index]=58000
data[' Income '].loc[data.loc[(data.Education=='Graduation') & (data[' Income '].isna())].index]=50000
data[' Income '].loc[data.loc[(data.Education=='Master') & (data[' Income '].isna())].index]=55000
data[' Income '].loc[data.loc[(data.Education=='2n Cycle') & (data[' Income '].isna())].index]=48000
clean_income=data[' Income '].str.replace('$', '').str.replace(',', '').astype('float')
data['clean_income']=clean_income
data.drop(' Income ', axis=1, inplace=True)
from sklearn.preprocessing import OneHotEncoder
Cat_c=['Education', 'Marital_Status','Country']
for items in Cat_c:
    le=OneHotEncoder()
    t=le.fit_transform(data[[items]]).toarray()
    a=data[items].unique()
    indexs=np.unique(a, return_index=True)[1]
    col=[a[indexs] for index in sorted(indexs)]
    data=pd.concat([data, pd.DataFrame(t, columns=col[1])], axis=1)
data['Dt_Customer_n']=pd.to_datetime(data.Dt_Customer)
data=data.drop(['Dt_Customer'], axis=1)
data=data.drop(index=527)
data=data.drop(index=[513, 827, 2233])
data=data.drop(index=data[data.clean_income>150000].index)

We are trying to do the following tasks in this notebook.

# Section 02: Statistical Analysis
Please run statistical tests or regressions to answer these questions. Make sure to interpret your results with non-statistical jargon so your CMO can understand your findings.

What factors are significantly related to the number of store purchases?
Does US fare significantly better than the Rest of the World in terms of total purchases?
Your supervisor insists that people who buy gold are more conservative. Therefore, people who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test
Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? What other factors are significantly related to amount spent on fish? (Hint: use your knowledge of interaction variables/effects)
Is there a significant relationship between geographical regional and success of a campaign?

In [ ]:
pd.set_option('display.max_columns', 50)
data.head(10)

1. What factors are significantly related to the number of store purchases?

In [ ]:
data.describe().NumStorePurchases

In [ ]:
plt.hist(data.NumStorePurchases,color = 'blue')
plt.xlabel('number of store purchases ')
plt.ylabel('Quantity')

We do not see a significant outlier in the number of store purchases data. We are also dealing with a left-skewed distribution. If we want to use a model that assumes the imput data to be normally disturbed, we may want to normalize the data. However, we are good for now. 

In [ ]:
data.corr().NumStorePurchases.hist()

As can be seen from above graph, most of the features has almost no effect on the number of store purchases (they are around zero), and obviously 1.0 is the "NumStorePurchases" which is the number pf store purchases. We have a few features with a very negative effet (around -0.5) and a couple features with high positive correlation with "NumStorePurchases" (btw 0.4-0.7). Lets check these features:  

In [ ]:
data.corr().loc[(data.corr().NumStorePurchases<-0.3) | (data.corr().NumStorePurchases>0.4)]['NumStorePurchases']

off course if you have more income, you are gooing to have more number of purchases made directly in stores :). 
So, if you have more kidhome, you are going to have a less number of purchases made directly in stores? It somehow makes sense, as it is difficult to go to the stores with kids. Maybe the web purchase will have a direct correlation with number of kids at home. As can bee seen, if the number of web visits increases, the number of store prchases decreases.

Other positive effects on the number of store visits are the number of different goods that a famility is purchasing. Obviously, when you need more goods you go to the store more often.  

Anyway, lets have a closer look to the most effective paramets on NumStorePurchases:

In [ ]:
sns.lmplot(x='NumStorePurchases', y='clean_income', data=data)


In [ ]:
fig, axs=plt.subplots(2,2, figsize=(10,8))
sns.boxplot(x='Kidhome', y='NumStorePurchases', data=data, ax=axs[0,0])
sns.boxplot(x='Kidhome', y='NumWebVisitsMonth', data=data, ax=axs[0,1])
sns.boxplot(x=data['Kidhome'], y=data['NumStorePurchases']+data['NumWebVisitsMonth'], ax=axs[1,0])
sns.boxplot(y=data['MntWines'], x=data['NumStorePurchases'], ax=axs[1,1])


As expected, families with no kids prefer to go to the store for purchasing and they are not want to shop online. On ther other hand, families with one or two kids shopping online more often than no kids families. In general, the total buying from store and web for all families are almost the same (figure 3)
Another intersting thing is the number of wines buying and the amount of store purchasing. The figure 4 illustrates that families whom buying more wines, visiting store more often. Good for liquor stores :)

2. Does US fare significantly better than the Rest of the World in terms of total purchases? 

Okay, great question. Do we have a total purchases column? It seems no, so we need to generate this column:

In [ ]:
data.columns

In [ ]:
purchase=data[['MntWines','MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases','NumCatalogPurchases']]
data['Mnt_total_purchases']=purchase.sum(axis=1)

In [ ]:
sns.boxplot(x='Country', y='Mnt_total_purchases', data=data)

As can be seen, the toral amount of purchases in almost the same for all countries except "ME"

3. Your supervisor insists that people who buy gold are more conservative. Therefore, people who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test.

In [ ]:
sns.lineplot(y='MntGoldProds', x='NumStorePurchases', data=data)


It seems a large number of gold buyer have a less store buying! Most of them only go to store once, which may be for gold buying :) Ignoring one store purchases, by increasing the number of stor purchases, money spend for gold increases, but it does not seems to be related to being "conservative"; as previously discussed, the high number of store purchases is direcly related to the income amount as well. Hence, more income ==> more gold 


4. Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? What other factors are significantly related to amount spent on fish? 

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x=data['Education'], y=data['MntFishProducts'], hue=data['Marital_Status'])

As can be seen, PhDs fish spend does not have a significant different with other educated persons. For a Basic eduction, fish spending is very low. It seems "widow" group spending more money on fish regardless of education. 

Lets check the relation btw fish spending and country:

In [ ]:
sns.lineplot(x=data['Country'], y=data['MntFishProducts'])

Almost all countries has a same amount of spending on Fish, except "ME" 